In [82]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from time import time 
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import seaborn as sns
%matplotlib inline

In [83]:
# Load training data as dataframe
df = pd.read_csv('final_train.csv')

display(df)

,Unnamed: 0,Activity,"angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyAccMean,gravity)","angle(tBodyGyroJerkMean,gravityMean)","angle(tBodyGyroMean,gravityMean)",energy-mean(),...,tGravityAccMag-energy(),tGravityAccMag-entropy(),tGravityAccMag-iqr(),tGravityAccMag-mad(),tGravityAccMag-max(),tGravityAccMag-mean(),tGravityAccMag-min(),tGravityAccMag-sma(),tGravityAccMag-std(),void()
0,0,STANDING,-1.574191,-0.805700,-2.453831,-1.861714,0.115517,-2.023369,3.125860,-9.474436,...,5.049130,3.272281,5.103018,-0.823302,3.063681,-0.928676,-1.007695,3.094236,1.669987,1.0
1,1,WALKING_UPSTAIRS,-1.568669,-0.612620,-2.488338,-1.751117,0.218134,0.280294,7.953403,-0.679894,...,4.565443,4.440779,3.288399,-2.689605,1.705341,-0.822617,-0.572410,-2.187890,-0.544652,2.0
2,2,LAYING,-1.548333,-0.139628,-2.380889,-1.915239,0.067613,-0.887342,6.007361,-11.906904,...,5.059609,2.296603,5.344174,-0.549843,3.272225,-0.940610,-1.031522,-2.071755,2.024442,3.0
3,3,WALKING,-1.574094,-0.793443,-2.449959,-1.842741,-0.195625,-0.950671,2.104353,-11.906121,...,4.860758,4.177535,3.950012,-0.322642,2.366287,-0.866937,-0.696431,1.863375,0.550414,2.0
4,4,LAYING,-1.548333,-0.139628,-2.380889,NaN,0.067613,-0.887342,6.007361,-11.970485,...,5.059609,2.296603,5.344174,-0.549843,3.272225,-0.940610,-1.031522,-2.102513,2.024442,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11081,11081,WALKING_UPSTAIRS,-1.568669,-0.612620,-2.488338,-1.751117,0.218134,0.280294,7.953403,-0.679894,...,4.565443,4.440779,3.288399,-2.689605,1.705341,-0.822617,-0.572410,3.534816,-0.544652,2.0
11082,11082,LAYING,-1.550288,-0.231565,-2.376561,-1.868524,0.007653,-0.077020,5.494466,-5.860692,...,5.048737,2.923856,5.182026,-0.998495,2.853755,-0.933561,-1.000173,-1.984575,1.372763,1.0
11083,11083,WALKING_UPSTAIRS,-1.569815,-0.794940,-2.482325,-1.784151,0.021844,-0.740329,8.005261,-11.916107,...,4.453514,4.302852,2.979961,-6.032382,1.853029,-0.802817,0.041975,3.427548,-0.075184,1.0
11084,11084,WALKING_DOWNSTAIRS,-1.574768,-0.752847,-2.449529,-1.737755,-0.189761,-0.454246,8.054443,-5.611277,...,4.615939,4.443277,3.208597,-6.200222,1.675269,-0.828934,-0.877004,3.385405,-0.383217,3.0


In [84]:
# Correct the order of values and label
df_analysis = df[[
    'angle(X,gravityMean)',
'angle(Y,gravityMean)',
'angle(Z,gravityMean)',
'angle(tBodyAccJerkMean),gravityMean)',
'angle(tBodyAccMean,gravity)',
'angle(tBodyGyroJerkMean,gravityMean)',
'angle(tBodyGyroMean,gravityMean)',
'energy-mean()',
'fBodyAcc-energy()-X',
'fBodyAcc-energy()-Y',
'fBodyAcc-energy()-Z',
'fBodyAcc-max()-X',
'fBodyAcc-max()-Y',
'fBodyAcc-max()-Z',
'Activity'
]]

display(df_analysis)

,"angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyAccMean,gravity)","angle(tBodyGyroJerkMean,gravityMean)","angle(tBodyGyroMean,gravityMean)",energy-mean(),fBodyAcc-energy()-X,fBodyAcc-energy()-Y,fBodyAcc-energy()-Z,fBodyAcc-max()-X,fBodyAcc-max()-Y,fBodyAcc-max()-Z,Activity
0,-1.574191,-0.805700,-2.453831,-1.861714,0.115517,-2.023369,3.125860,-9.474436,6.100845,-0.542301,-6.472294,5.242441,6.252482,0.495253,STANDING
1,-1.568669,-0.612620,-2.488338,-1.751117,0.218134,0.280294,7.953403,-0.679894,5.164649,-0.213678,-6.389757,5.297821,5.299352,0.455411,WALKING_UPSTAIRS
2,-1.548333,-0.139628,-2.380889,-1.915239,0.067613,-0.887342,6.007361,-11.906904,6.102219,-0.563555,-6.474598,5.244106,6.606042,0.510324,LAYING
3,-1.574094,-0.793443,-2.449959,-1.842741,-0.195625,-0.950671,2.104353,-11.906121,5.874761,-0.174192,-6.446539,5.276666,5.438051,0.462394,WALKING
4,-1.548333,-0.139628,-2.380889,NaN,0.067613,-0.887342,6.007361,-11.970485,6.102219,-0.563555,-6.474598,5.244106,6.606042,0.510324,LAYING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11081,-1.568669,-0.612620,-2.488338,-1.751117,0.218134,0.280294,7.953403,-0.679894,5.164649,-0.213678,-6.389757,5.297821,5.299352,0.455411,WALKING_UPSTAIRS
11082,-1.550288,-0.231565,-2.376561,-1.868524,0.007653,-0.077020,5.494466,-5.860692,6.087379,-0.545357,-6.473801,5.248606,6.261841,0.506342,LAYING
11083,-1.569815,-0.794940,-2.482325,-1.784151,0.021844,-0.740329,8.005261,-11.916107,5.324136,0.059691,-6.350640,5.310486,4.539793,0.409625,WALKING_UPSTAIRS
11084,-1.574768,-0.752847,-2.449529,-1.737755,-0.189761,-0.454246,8.054443,-5.611277,5.208765,0.041371,-6.452305,5.333956,4.924305,0.461533,WALKING_DOWNSTAIRS


In [85]:
variables = [
    'angle(X,gravityMean)',
'angle(Y,gravityMean)',
'angle(Z,gravityMean)',
'angle(tBodyAccJerkMean),gravityMean)',
'angle(tBodyAccMean,gravity)',
'angle(tBodyGyroJerkMean,gravityMean)',
'angle(tBodyGyroMean,gravityMean)',
'energy-mean()',
'fBodyAcc-energy()-X',
'fBodyAcc-energy()-Y',
'fBodyAcc-energy()-Z',
'fBodyAcc-max()-X',
'fBodyAcc-max()-Y',
'fBodyAcc-max()-Z']

In [86]:
X_values = df_analysis[variables]
Y_labels = df_analysis['Activity']

In [87]:
X_values.describe()

,"angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyAccMean,gravity)","angle(tBodyGyroJerkMean,gravityMean)","angle(tBodyGyroMean,gravityMean)",energy-mean(),fBodyAcc-energy()-X,fBodyAcc-energy()-Y,fBodyAcc-energy()-Z,fBodyAcc-max()-X,fBodyAcc-max()-Y,fBodyAcc-max()-Z
count,11029.000000,11018.000000,11026.000000,11034.000000,11034.000000,11029.000000,11030.000000,11026.000000,11030.000000,11038.000000,11027.000000,11029.000000,11030.000000,11033.000000
mean,-1.566818,-0.615862,-2.446177,-1.860175,0.090448,-0.156444,5.328109,-7.711158,5.583336,-0.301356,-6.446143,5.276721,5.844693,0.482470
std,0.009901,0.330366,0.033629,0.070504,0.238381,1.117164,1.969548,4.329267,0.743518,0.351643,0.038643,0.043104,0.866142,0.030914
min,-1.577122,-1.839413,-2.562914,-2.012415,-0.702915,-2.689919,1.772903,-12.306239,-0.221561,-0.564249,-6.474698,5.240813,2.120900,0.336754
25%,-1.573312,-0.850689,-2.467732,-1.915239,0.025197,-0.887342,3.763255,-11.912809,5.164649,-0.563555,-6.474598,5.241648,5.209530,0.455411
50%,-1.570783,-0.726956,-2.453059,-1.865249,0.076989,-0.127773,5.656617,-7.267033,6.101476,-0.554946,-6.473443,5.244252,6.389979,0.499976
75%,-1.567797,-0.468971,-2.436508,-1.809428,0.218134,0.593004,6.919413,-4.583362,6.103856,-0.120657,-6.422122,5.302643,6.606042,0.510002
max,-1.536682,0.720197,-2.345198,-1.714677,0.845884,2.393603,8.247982,3.510343,6.104109,1.790725,-6.146734,5.467578,6.657819,0.511408


In [88]:
Y_labels.describe()

count      11086
unique         6
top       LAYING
freq        2167
Name: Activity, dtype: object

In [89]:
# Standardising the data
# Center to the mean and component wise scale to unit variance
# for col in [variables]:
#     X_all[col] = scale(X_all[col])
sc = StandardScaler()
X_values = sc.fit_transform(X_values)

In [90]:
# Dividing data into training and test set taking 80% of prior data as a training set,
# so that our models are tested based on matches that took place after matches from our training set
X_train, X_test= np.split(X_values, [int(.8 *len(X_values))])
Y_train, Y_test= np.split(Y_labels, [int(.8 *len(Y_labels))])

In [91]:
# Printing shape of our data sets
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', Y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', Y_test.shape)

Training Features Shape: (8868, 14)
Training Labels Shape: (8868,)
Testing Features Shape: (2218, 14)
Testing Labels Shape: (2218,)


In [92]:
# Training the classifier and measuring its time
def train(clf, X_train, Y_train):
    
    # Starting the clock, training the classifier, then stoping the clock
    start = time()
    clf.fit(X_train, Y_train)
    end = time()
    
    # Printing the results
    print ("Model trained in: ",end - start)

In [93]:
# Making predictions 
def predict(clf, features, target):
    
    # Starting the clock, making predictions, then stoping the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Printing results
    print ("Prediction made in: ",end - start)
    
    # Returning F1 score and accuracy
    return f1_score(target, y_pred, average=None), sum(target == y_pred) / float(len(y_pred))

In [94]:
# Evaluation of our model
def fit(clf, X_train, y_train, X_test, y_test):
    
    # Indicating the model and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Training model
    train(clf, X_train, y_train)
    
    # Printing the results of prediction for both training and testing set
    f1, acc = predict(clf, X_train, y_train)
    print ("===========================================")
    print ("Indexes assesing methods for training set:")
    print ("F1 score [LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS]:", f1)
    print ("Accuracy:", acc)
    
    print ("===========================================")
    f1, acc = predict(clf, X_test, y_test)
    print ("Indexes assesing methods for test set:")
    print ("F1 score [LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS]:", f1)
    print ("Accuracy:", acc)

In [101]:
# Inference function
def inference(clf_XGB, inf):
    start = time()
    y_pred = clf_XGB.predict(inf)
    end = time()

    # Printing results
    print ("Inference made in: ",end - start)

In [98]:
# Creating our model and training it
clf_XGB = xgb.XGBClassifier(seed=66)

fit(clf_XGB, X_train, Y_train, X_test, Y_test)
print ('')

Training a XGBClassifier using a training set size of 8868. . .
[23:38:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Model trained in:  5.265996694564819
Prediction made in:  0.04500007629394531
Indexes assesing methods for training set:
F1 score [LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS]: [1. 1. 1. 1. 1. 1.]
Accuracy: 1.0
Prediction made in:  0.010999679565429688
Indexes assesing methods for test set:
F1 score [LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS]: [1.         0.94451004 0.94039735 0.95324675 0.93785311 0.96995708]
Accuracy: 0.9589720468890893



In [102]:
# Inference time
pred_try = np.array([X_train[0]])
inference(clf_XGB, pred_try)

Inference made in:  0.003999471664428711
